In [17]:
import numpy as np
import imageio.v2 as imageio

In [18]:
# Read the .bmp input and output files as a numpy 2D matrix
X = imageio.imread('x1.bmp')
# Convert the image to a NumPy array
X = np.array(X)

Y = imageio.imread('y4.bmp')
Y = np.array(Y)

print('X shape:', X.shape)
print('Y shape:', Y.shape)

X shape: (140, 188)
Y shape: (256, 188)


In [19]:
def isPsevdoInversed(A, A_psevdo_inverse) -> bool:
    #  A * A+ * A = A;
    if not np.allclose(A @ A_psevdo_inverse @ A, A):
        print('A * A+ * A != A')
        return False
    #  A+ * A * A+ = A+
    elif not np.allclose(A_psevdo_inverse @ A @ A_psevdo_inverse, A_psevdo_inverse):
        print('A+ * A * A+ != A+')
        return False
    #  A * A+ - symmetric matrix m x m
    elif not np.allclose(A @ A_psevdo_inverse, (A @ A_psevdo_inverse).T):
        print('A * A+ - not symmetric matrix m x m')
        return False
    #  A+ * A - symmetric matrix n x n
    elif not np.allclose(A_psevdo_inverse @ A, (A_psevdo_inverse @ A).T):
        print('A+ * A - not symmetric matrix n x n')
        return False
    
    return True

In [20]:
def PseudoInverseMatrix_MoorePenrose(A, eps=1e-6, delta=5):
    # Calculate the Moore-Penrose pseudo-inverse of a matrix
    # A: input matrix
    # returns: pseudo-inverse of A
    # eps: tolerance for zero singular values
    print('delta:', delta)

    A0 = A.T @ np.linalg.inv(A @ A.T + delta**2 * np.eye(A.shape[0]))
    print('A0:', A0)
    
    delta = delta / 2
    print('delta:', delta)

    while True:    
        A1 = A.T @ np.linalg.inv(A @ A.T + delta**2 * np.eye(A.shape[0]))
        print('A1:', A1)
        if np.linalg.norm(A0 - A1, ord=2) < eps:
            return A1
        
        delta = delta / 2
        print('delta:', delta)

        A0 = A1

# Test
Atest = np.array([[1, 2, 3], [4, 5, 6]])
Atest_pinv = PseudoInverseMatrix_MoorePenrose(Atest)
print('Atest:', Atest)
print('Atest_pinv:', Atest_pinv)
assert isPsevdoInversed(Atest, Atest_pinv)

delta: 5
A0: [[-0.00880162  0.04197698]
 [ 0.01489506  0.04434665]
 [ 0.03859174  0.04671632]]
delta: 2.5
A1: [[-0.06761734  0.0740391 ]
 [ 0.00982151  0.05628482]
 [ 0.08726036  0.03853055]]
delta: 1.25
A1: [[-0.24889378  0.15229405]
 [-0.01447423  0.06953922]
 [ 0.21994533 -0.0132156 ]]
delta: 0.625
A1: [[-0.56421046  0.28497941]
 [-0.05818039  0.08866413]
 [ 0.44784967 -0.10765115]]
delta: 0.3125
A1: [[-0.80930643  0.38779137]
 [-0.09228987  0.10315847]
 [ 0.62472668 -0.18147444]]
delta: 0.15625
A1: [[-0.90667952  0.42861403]
 [-0.10585078  0.10889048]
 [ 0.69497795 -0.21083308]]
delta: 0.078125
A1: [[-0.93471669  0.44036684]
 [-0.10975608  0.11053922]
 [ 0.71520454 -0.2192884 ]]
delta: 0.0390625
A1: [[-0.94199391  0.44341726]
 [-0.11076976  0.11096705]
 [ 0.72045439 -0.22148315]]
delta: 0.01953125
A1: [[-0.94383064  0.44418716]
 [-0.11102561  0.11107503]
 [ 0.72177942 -0.2220371 ]]
delta: 0.009765625
A1: [[-0.94429092  0.44438009]
 [-0.11108973  0.11110209]
 [ 0.72211147 -0.2221759

In [21]:
def PseudoInverseMatrix_Greville(A, eps=1e-6):
    # Calculate the Greville pseudo-inverse of a matrix
    # A: input matrix
    # returns: pseudo-inverse of A
    # eps: tolerance for zero singular values
    inverse_matrix = np.vstack(A[0] / np.dot(A[0].T, A[0]) if np.dot(A[0].T, A[0]) != 0 else A[0])
    current_matrix = np.array([A[0]])
    n = A.shape[0]
    for i in range(1, n):
        a = A[i].reshape(-1, 1)
        z = np.identity(current_matrix.shape[1]) - np.dot(inverse_matrix, current_matrix)
        current_matrix = np.vstack([current_matrix, A[i]])

        denum = np.dot(a.T, np.dot(z, a))[0, 0]
        if np.abs(denum) < eps:
            r = np.dot(inverse_matrix, inverse_matrix.T)
            denum = 1 + np.dot(a.T, np.dot(r, a))
            inverse_matrix = np.hstack((inverse_matrix - np.dot(z, np.dot(a, np.dot(a.T, inverse_matrix))) / denum, np.dot(r, a) / denum))
        else:
            inverse_matrix = np.hstack((inverse_matrix - np.dot(z, np.dot(a, np.dot(a.T, inverse_matrix))) / denum, np.dot(z, a) / denum))

        print('-------------------------------------------------------------')
        print('inverse_matrix:', inverse_matrix)

    return inverse_matrix

# Test
Atest = np.array([[1, 2, 3], [4, 5, 6]])
Atest_pinv = PseudoInverseMatrix_Greville(Atest)
print('Atest:', Atest)
print('Atest_pinv:', Atest_pinv)
assert isPsevdoInversed(Atest, Atest_pinv)

-------------------------------------------------------------
inverse_matrix: [[-0.94444444  0.44444444]
 [-0.11111111  0.11111111]
 [ 0.72222222 -0.22222222]]
Atest: [[1 2 3]
 [4 5 6]]
Atest_pinv: [[-0.94444444  0.44444444]
 [-0.11111111  0.11111111]
 [ 0.72222222 -0.22222222]]


In [22]:
def CalculateOperator(X, Y, inversion_function, V=np.ones(10, 10), eps=1e-6):
    X_pinv = inversion_function(X, eps=eps)
    YX_pinv = Y @ X_pinv
    VZTXT = V @ (np.eye(X.shape[1]) - X & X_pinv).T
    A = YX_pinv @ VZTXT
    return A